<a href="https://colab.research.google.com/github/lancui1110/lihongyi_deepLearning/blob/main/HW01/HW01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Homework 1: COVID-19 Cases Prediction (Regression)

Slides: [Slides: https://github.com/ga642381/ML2021-Spring/blob/main/HW01/HW01.pdf](https://github.com/ga642381/ML2021-Spring/blob/main/HW01/HW01.pdf)

In [2]:
tr_path = 'covid.train.csv'
tt_path = 'covid.test.csv'

!gdown --id '19CCyCgJrUxtvgZF53vnctJiOJ23T5mqF' --output covid.train.csv
!gdown --id '1CE240jLm2npU-tdz81-oVKEF3T2yfT1O' --output covid.test.csv

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=19CCyCgJrUxtvgZF53vnctJiOJ23T5mqF
To: /content/covid.train.csv
100% 2.00M/2.00M [00:00<00:00, 140MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1CE240jLm2npU-tdz81-oVKEF3T2yfT1O
To: /content/covid.test.csv
100% 651k/651k [00:00<00:00, 56.8MB/s]


In [5]:
#PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# For data process 数据程序
import numpy as np
import csv
import os

# For plotting 绘图
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

myseed = 42069 # set a ramdom seed for reproducibility 为再现性设置一个随机种子
# 配置了PyTorch的cudnn后端
torch.backends.cudnn.deterministic = True # 为了确保每次运行代码时，卷积操作的结果都是相同的
torch.backends.cudnn.benchmark = False # 关闭运行时的算法自动选择功能，这样可以保证每次运行时使用的卷积算法是相同的，有助于结果的可重复性
np.random.seed(myseed) # 设置了numpy的随机种子，确保numpy生成的随机数是可重复的
torch.manual_seed(myseed) # 设置了PyTorch的随机种子，确保PyTorch生成的随机数是可重复的
if torch.cuda.is_available(): # 检查是否有可用的CUDA设备（通常是NVIDIA的GPU）
    torch.cuda.manual_seed_all() # 为所有的CUDA设备设置随机种子，确保在多GPU环境下也能获得可重复的结果

In [ ]:
def get_device():
  ''' get device (if GPU is available, use GPU) '''
  return 'cuda' if torch.cuda.is_available else 'cpu'

def plot_learning_curve(loss_record, title=''):
  '''plot lerning curve of your DNN (train and dev loss)'''